# Read in HTMLs of SEC Filings

## Imports

In [117]:
import glob
import json

import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

import re

import spacy

## Create function to extract stock from filepath

In [127]:
def extract_stock(string, filepath = True):
    if filepath:
        return re.search(r'data\\([a-z]+)[-\d_]',string).group(1)
    else:
        return re.search(r'^([a-z]+)[-\d_]',string).group(1)

## Make soups dictionary

In [96]:
soups = {extract_stock(filepath): BS(open(filepath), 'html.parser') for filepath in glob.glob('../data/*.htm*')}

## Create text-creation function and texts dictionary

In [108]:
 def make_text(stock):
    text_list = [tag.get_text().strip() for tag in soups[stock].find_all(['div', 'p'])]
    text = '\n'.join(text_list).replace('\xa0', ' ').replace('  ', ' ')
    return text

In [111]:
texts_dict = {stock: make_text(stock) for stock in soups.keys()}

In [114]:
texts = texts_dict.values()
stocks = texts_dict.keys()

## Read in annotations

In [126]:
texts_dict.keys()

dict_keys(['amrk', 'apog', 'azz', 'bbw', 'cnxc', 'ctrn', 'earn', 'fnhc', 'ful', 'gwre', 'info', 'intc', 'kbh', 'kmx', 'lake', 'lvis', 'mcft', 'mdc', 'mtn', 'nflx', 'noc', 'panw', 'prgs', 'pstg', 'siri', 'slb', 'tlys', 'tmb', 'uri', 'vrrm'])

In [119]:
annotations = json.load(open('../data/annotations.json'))

In [125]:
annotations[0]['filing']

'azz-20220228.html'

In [156]:
extract_dict = {}
for entry in annotations:
    stock = extract_stock(entry['filing'], filepath=False)
    text = entry['text']
    if stock in extract_dict.keys():
        extract_dict[stock][text] = {key:value for key, value in entry.items() if key not in ['filing', 'text']}
    else:
        extract_dict[stock] = {}
        extract_dict[stock][text] = {key:value for key, value in entry.items() if key not in ['filing', 'text']}

In [157]:
list(extract_dict['intc'].keys())[0]

'In the first quarter of 2021, we repurchased the remaining $2.4 billion in shares of our planned $20.0 billion share repurchases announced in October 2019. We expect our future stock repurchases to be significantly below our levels from the last few years.'

## Generate spacy model, documentize texts, and add metadata

In [ ]:
texts_dict

In [173]:
data = {'repurchase': [annotation['text'] for annotation in annotations],
        'irrelevant': []}

In [180]:
from spacy.pipeline.textcat import DEFAULT_SINGLE_TEXTCAT_MODEL
import classy_classification

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("text_categorizer",
              config = {"model": 'spacy',
                      "data": data
                      }
        )

Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\jrior\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\jrior\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jrior\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 199, in fit
    y = self._validate_targets(y)
  File "C:\Users\jrior\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 720, in _validate_targets
    raise ValueError(
ValueError: The number of classes has to be greater than o

ValueError: The number of classes has to be greater than one; got 1 class